#### Model Inference

In [ ]:
import re
import numpy as np
import pandas as pd

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [5]:
class GdDataset(Dataset):
    def __init__(self, tokenizer, prefix, dataframe, max_len=512):
        self.tokenizer = tokenizer
        self.prefix = prefix
        self.dataframe = dataframe
        self.max_len =  max_len

        self.inputs = []
        self._build()

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        input_ids = self.inputs[index]["input_ids"].squeeze()
        attention_mask = self.inputs[index]["attention_mask"].squeeze()

        return {"input_ids":input_ids, "attention_mask":attention_mask}

    def _build(self):
        for row in self.dataframe.itertuples(index=False):
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [self.prefix + row.text], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
                )
            self.inputs.append(tokenized_inputs)
        

In [6]:
data = pd.read_csv("agg.csv")
dataframe = data[['instance_id', 'text', 'man_score', 'woman_score',
                  'white_score', 'black_score', 'asian_score', 'nonwhite_score',
                  'offensiveness_label']]

In [6]:
data = pd.read_csv("politeness_1500.csv")
dataframe = data[['instance_id', 'text', 'man_score', 'woman_score',
                  'white_score', 'black_score', 'asian_score', 'nonwhite_score',
                  'label']]

In [25]:
dataframe = pd.read_csv("offensiveness_results.csv")

In [26]:
prefix = "How do you rate how offensive the sentence is from 1 (not offensive at all) to 5 (very offensive). Options: 1, 2, 3, 4, 5: "
dataset = GdDataset(tokenizer, prefix, dataframe)
loader = DataLoader(dataset, batch_size=32)

/opt/anaconda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2398: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
outputs = []

for batch in tqdm(loader):
    outs = model.generate(input_ids=batch["input_ids"].cuda(),
                          attention_mask=batch["attention_mask"].cuda())
    dec = [tokenizer.decode(ids) for ids in outs]
    outputs.extend(dec)

  0%|          | 0/47 [00:00<?, ?it/s]/opt/anaconda/lib/python3.9/site-packages/transformers/generation/utils.py:1355: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 47/47 [02:33<00:00,  3.27s/it]


In [7]:
def func_1(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

In [28]:
dataframe['t5_base'] = [func_1(re.findall(r'<pad> (.*)</s>|$', i)[0]) for i in outputs]

In [30]:
dataframe.to_csv("offensiveness_results.csv", index=False)

##### Offensiveness

In [26]:
dataframe.to_csv("offensiveness_results.csv", index=False)

In [28]:
dataframe[['offensiveness_label', 'woman_score', 'man_score', 't5_female', 't5_male']].corr()

,offensiveness_label,woman_score,man_score,t5_female,t5_male
offensiveness_label,1.000000,0.886194,0.859597,0.572129,0.559087
woman_score,0.886194,1.000000,0.588440,0.508472,0.505532
man_score,0.859597,0.588440,1.000000,0.480209,0.457415
t5_female,0.572129,0.508472,0.480209,1.000000,0.944830
t5_male,0.559087,0.505532,0.457415,0.944830,1.000000


In [30]:
dataframe[['offensiveness_label', 'white_score', 'black_score', 'asian_score', 'nonwhite_score', 't5_white', 't5_black', 't5_asian', 't5_nwhite']].corr()

,offensiveness_label,white_score,black_score,asian_score,nonwhite_score,t5_white,t5_black,t5_asian,t5_nwhite
offensiveness_label,1.000000,0.951949,0.665420,0.607914,0.702225,0.532454,0.431274,0.477106,0.487406
white_score,0.951949,1.000000,0.459541,0.412191,0.485856,0.510847,0.413723,0.453439,0.462110
black_score,0.665420,0.459541,1.000000,0.313313,0.883433,0.336824,0.294883,0.306738,0.306076
asian_score,0.607914,0.412191,0.313313,1.000000,0.833277,0.281539,0.198625,0.255149,0.280688
nonwhite_score,0.702225,0.485856,0.883433,0.833277,1.000000,0.368690,0.288952,0.342738,0.349146
t5_white,0.532454,0.510847,0.336824,0.281539,0.368690,1.000000,0.680823,0.775075,0.844475
t5_black,0.431274,0.413723,0.294883,0.198625,0.288952,0.680823,1.000000,0.702795,0.681454
t5_asian,0.477106,0.453439,0.306738,0.255149,0.342738,0.775075,0.702795,1.000000,0.752964
t5_nwhite,0.487406,0.462110,0.306076,0.280688,0.349146,0.844475,0.681454,0.752964,1.000000


##### Politeness

In [27]:
dataframe.to_csv("politeness_results.csv", index=False)

In [28]:
dataframe[['label', 'woman_score', 'man_score', 't5_female', 't5_male']].corr()

,label,woman_score,man_score,t5_female,t5_male
label,1.000000,0.913951,0.871574,0.704577,0.696097
woman_score,0.913951,1.000000,0.661776,0.642404,0.629376
man_score,0.871574,0.661776,1.000000,0.621085,0.612807
t5_female,0.704577,0.642404,0.621085,1.000000,0.914906
t5_male,0.696097,0.629376,0.612807,0.914906,1.000000


In [30]:
dataframe[['label', 'white_score', 'black_score', 'asian_score', 'nonwhite_score', 't5_white', 't5_black', 't5_asian', 't5_nwhite']].corr()

,label,white_score,black_score,asian_score,nonwhite_score,t5_white,t5_black,t5_asian,t5_nwhite
label,1.000000,0.961691,0.734787,0.740195,0.799760,0.717389,0.699355,0.664146,0.670903
white_score,0.961691,1.000000,0.570726,0.607198,0.628572,0.699500,0.684615,0.651077,0.657932
black_score,0.734787,0.570726,1.000000,0.453552,0.897044,0.513196,0.503193,0.466550,0.482531
asian_score,0.740195,0.607198,0.453552,1.000000,0.854298,0.520972,0.510125,0.498755,0.490712
nonwhite_score,0.799760,0.628572,0.897044,0.854298,1.000000,0.549166,0.533041,0.503124,0.506105
t5_white,0.717389,0.699500,0.513196,0.520972,0.549166,1.000000,0.883490,0.789990,0.821255
t5_black,0.699355,0.684615,0.503193,0.510125,0.533041,0.883490,1.000000,0.847226,0.891406
t5_asian,0.664146,0.651077,0.466550,0.498755,0.503124,0.789990,0.847226,1.000000,0.870262
t5_nwhite,0.670903,0.657932,0.482531,0.490712,0.506105,0.821255,0.891406,0.870262,1.000000


##### Random Nouns Test

In [9]:
nouns = pd.read_csv("noun_list.csv", header=0)
noun_list = [i for i in nouns.loc[0:20, 'word']]

In [9]:
results = {}

for i in tqdm(noun_list):
    prefix = f"How does a {i} rate how offensive the sentence is from 1 (not offensive at all) to 5 (very offensive). Options: 1, 2, 3, 4, 5: "
    dataset = GdDataset(tokenizer, prefix, dataframe)
    loader = DataLoader(dataset, batch_size=32)

    outputs = []
    for batch in tqdm(loader):
        outs = model.generate(input_ids=batch["input_ids"].cuda(),
                              attention_mask=batch["attention_mask"].cuda())
        dec = [tokenizer.decode(ids) for ids in outs]
        outputs.extend(dec)
    
    results[i] = [func_1(re.findall(r'<pad> (.*)</s>|$', i)[0]) for i in outputs]

  0%|          | 0/21 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/anaconda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/anaconda/lib/python3.9/site-packages/transformers/gene

In [10]:
pd.DataFrame.from_dict(results).to_csv("offensiveness_base_random_results.csv", index=False)

In [11]:
data = pd.read_csv("politeness.csv")
dataframe = data[['instance_id', 'text', 'man_score', 'woman_score',
                  'white_score', 'black_score', 'asian_score', 'nonwhite_score',
                  'label']].sample(n=1500, replace=False)
dataframe.to_csv("politeness_1500.csv", index=False)

In [10]:
for j in ["male ", "female "]:
    results = {}

    for i in tqdm(noun_list):
        prefix = f"How does a {j}{i} rate how polite the sentence is from 1 (not polite at all) to 5 (very polite). Options: 1, 2, 3, 4, 5: "
        dataset = GdDataset(tokenizer, prefix, dataframe)
        loader = DataLoader(dataset, batch_size=32)

        outputs = []
        for batch in loader:
            outs = model.generate(input_ids=batch["input_ids"].cuda(),
                                attention_mask=batch["attention_mask"].cuda())
            dec = [tokenizer.decode(ids) for ids in outs]
            outputs.extend(dec)
        
        results[i] = [func_1(re.findall(r'<pad> (.*)</s>|$', i)[0]) for i in outputs]

    pd.DataFrame.from_dict(results).to_csv(f"politeness_{j}_random_results.csv", index=False)

  0%|          | 0/21 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/anaconda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2398: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/anaconda/lib/python3.9/site-packages/transformers/gene

In [19]:
dataframe['t5_scores'] = [re.findall(r'<pad> (.*)</s>|$', i)[0] for i in outputs]
dataframe['t5_scores'] = dataframe['t5_scores'].apply(func_1)

<ipython-input-19-0871b5857c55>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['t5_scores'] = [re.findall(r'<pad> (.*)</s>|$', i)[0] for i in outputs]
<ipython-input-19-0871b5857c55>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['t5_scores'] = dataframe['t5_scores'].apply(func_1)


In [30]:
dataframe[['instance_id', 'offensiveness_label', 'woman_score', 'man_score', 't5_scores']].to_csv("flant5_xl_offensiveness_test.csv", index=False)

In [22]:
dataframe[['offensiveness_label', 'white_score', 'black_score', 'asian_score', 'woman_score', 'man_score', 't5_scores']].corr()

,offensiveness_label,white_score,black_score,asian_score,woman_score,man_score,t5_scores
offensiveness_label,1.000000,0.951949,0.665420,0.607914,0.886194,0.859597,0.559087
white_score,0.951949,1.000000,0.459541,0.412191,0.834325,0.819814,0.546923
black_score,0.665420,0.459541,1.000000,0.313313,0.619013,0.546541,0.312050
asian_score,0.607914,0.412191,0.313313,1.000000,0.541024,0.536235,0.335882
woman_score,0.886194,0.834325,0.619013,0.541024,1.000000,0.588440,0.505532
man_score,0.859597,0.819814,0.546541,0.536235,0.588440,1.000000,0.457415
t5_scores,0.559087,0.546923,0.312050,0.335882,0.505532,0.457415,1.000000


In [21]:
dataframe["t5_scores"].mean()

2.618

In [21]:
del model

#### Correlation

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
def func_1(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

In [9]:
data = pd.read_csv("agg.csv")
dataframe = data[['instance_id', 'offensiveness_gold', 'text', 'scores', 'users', 'man_score',
                  'woman_score', 'white_score', 'black_score', 'asian_score', 'nonwhite_score',
                  'gender', 'scores_num', 'scores_std', 'offensiveness_label']]
data_all = pd.read_csv("flant5_xl_offensiveness.csv")
data_person = pd.read_csv("flant5_xl_offensiveness_p.csv")
data_white = pd.read_csv("flant5_xl_offensiveness_white.csv")
data_black = pd.read_csv("flant5_xl_offensiveness_black.csv")
data_asian = pd.read_csv("flant5_xl_offensiveness_asian.csv")
data_nwhite = pd.read_csv("flant5_xl_offensiveness_nwhite.csv")

dataframe_1 = dataframe[['instance_id', 'text', 'offensiveness_label', 'white_score', 'black_score', 'asian_score', 'nonwhite_score']]

dataframe_1['t5_all'] = data_all['t5_scores'].apply(func_1)
dataframe_1['t5_person'] = data_person['t5_scores'].apply(func_1)
dataframe_1['t5_white'] = data_white['t5_scores'].apply(func_1)
dataframe_1['t5_black'] = data_black['t5_scores'].apply(func_1)
dataframe_1['t5_asian'] = data_asian['t5_scores'].apply(func_1)
dataframe_1['t5_nwhite'] = data_nwhite['t5_scores'].apply(func_1)

<ipython-input-9-56d6734a7c11>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_1['t5_all'] = data_all['t5_scores'].apply(func_1)
<ipython-input-9-56d6734a7c11>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_1['t5_person'] = data_person['t5_scores'].apply(func_1)
<ipython-input-9-56d6734a7c11>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [11]:
dataframe_1.rename(columns={"offensiveness_label":"all_score"}).to_csv("flant5_xl_offensiveness_ethnicity.csv", index=False)

In [12]:
dataframe_1.drop(['instance_id', 'text'], axis=1).corr()

,offensiveness_label,white_score,black_score,asian_score,nonwhite_score,t5_all,t5_person,t5_white,t5_black,t5_asian,t5_nwhite
offensiveness_label,1.000000,0.951949,0.665420,0.607914,0.702225,0.419916,0.307335,0.176328,0.163355,0.149649,0.155313
white_score,0.951949,1.000000,0.459541,0.412191,0.485856,0.391969,0.285608,0.168220,0.159977,0.143175,0.150031
black_score,0.665420,0.459541,1.000000,0.313313,0.883433,0.266330,0.230419,0.131767,0.139887,0.110495,0.099941
asian_score,0.607914,0.412191,0.313313,1.000000,0.833277,0.279010,0.181890,0.131149,0.050126,0.051511,0.119543
nonwhite_score,0.702225,0.485856,0.883433,0.833277,1.000000,0.309465,0.245027,0.146192,0.108549,0.109905,0.110638
t5_all,0.419916,0.391969,0.266330,0.279010,0.309465,1.000000,0.778491,0.506982,0.350188,0.445223,0.426360
t5_person,0.307335,0.285608,0.230419,0.181890,0.245027,0.778491,1.000000,0.527179,0.328732,0.470252,0.429291
t5_white,0.176328,0.168220,0.131767,0.131149,0.146192,0.506982,0.527179,1.000000,0.567880,0.690999,0.774324
t5_black,0.163355,0.159977,0.139887,0.050126,0.108549,0.350188,0.328732,0.567880,1.000000,0.578954,0.654904
t5_asian,0.149649,0.143175,0.110495,0.051511,0.109905,0.445223,0.470252,0.690999,0.578954,1.000000,0.686525


In [2]:
pd.read_csv("flant5_xl_offensiveness_gender.csv").drop(['instance_id', 'text'], axis=1).corr()

,all_score,woman_score,man_score,t5_all,t5_person,t5_female,t5_male,t5_woman,t5_man
all_score,1.000000,0.886194,0.859597,0.419916,0.307335,0.288212,0.372686,0.241227,0.330948
woman_score,0.886194,1.000000,0.588440,0.371136,0.279721,0.244024,0.330173,0.202124,0.291693
man_score,0.859597,0.588440,1.000000,0.374351,0.268690,0.259497,0.326503,0.210024,0.288249
t5_all,0.419916,0.371136,0.374351,1.000000,0.778491,0.677945,0.781278,0.624696,0.742642
t5_person,0.307335,0.279721,0.268690,0.778491,1.000000,0.676446,0.769406,0.630969,0.748259
t5_female,0.288212,0.244024,0.259497,0.677945,0.676446,1.000000,0.794348,0.893926,0.826811
t5_male,0.372686,0.330173,0.326503,0.781278,0.769406,0.794348,1.000000,0.748793,0.888172
t5_woman,0.241227,0.202124,0.210024,0.624696,0.630969,0.893926,0.748793,1.000000,0.784709
t5_man,0.330948,0.291693,0.288249,0.742642,0.748259,0.826811,0.888172,0.784709,1.000000


#### Random noun test

In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np

In [2]:
white = pd.read_csv("white_random_results.csv")
black = pd.read_csv('black_random_results.csv')

In [4]:
white_person = pd.read_csv("flant5_xl_offensiveness_ethnicity.csv")

In [5]:
white.mean()

pencil       1.308000
phone        1.124667
computer     1.195333
chair        1.324000
table        1.295333
carpet       1.182000
bookshelf    1.252000
lamp         1.172000
clock        1.270667
window       1.146000
cat          1.225333
dog          1.080000
horse        1.254667
bird         1.129333
fish         1.098667
snake        1.248000
rabbit       1.236000
elephant     1.181333
squirrel     1.194667
fox          1.212667
school       1.136667
dtype: float64

In [8]:
black.mean()

pencil       1.357333
phone        1.081333
computer     1.081333
chair        1.228000
table        1.197333
carpet       1.222000
bookshelf    1.272000
lamp         1.180000
clock        1.236000
window       1.118667
cat          1.204000
dog          1.174667
horse        1.386667
bird         1.112000
fish         1.094667
snake        1.204000
rabbit       1.176000
elephant     1.208000
squirrel     1.134667
fox          1.178667
school       1.109333
dtype: float64

In [7]:
white_person['t5_white'].mean()

1.3086666666666666

In [3]:
white

,pencil,phone,computer,chair,table,carpet,bookshelf,lamp,clock,window,...,dog,horse,bird,fish,snake,rabbit,elephant,squirrel,fox,school
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1496,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1497,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1498,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
stats.ttest_ind(white.loc[0], black.loc[0], equal_var=False)

<ipython-input-7-e18fb4b62e0b>:1: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  stats.ttest_ind(white.loc[0], black.loc[0], equal_var=False)


Ttest_indResult(statistic=-1.0000000000000009, pvalue=0.32925657717170853)

In [6]:
type(white.loc[0])

pandas.core.series.Series

#### Supplement

In [ ]:
t5_all = [func_1(x) for x in dataframe_1['t5_all']]
t5_f = [func_1(x) for x in dataframe_1['t5_female']]
t5_m = [func_1(x) for x in dataframe_1['t5_male']]

score_all = []
score_f = []
score_m = []

for row in dataframe_1.itertuples():
    row_s = row.scores[1:-1]
    s = [float(x.strip()) for x in row_s.split(',')]
    row_g = row.gender[1:-1]
    g = [x.strip() for x in row_g.split(',')]
    s_f = [s[i] for i in range(len(s)) if g[i]=="'Woman'"]
    s_m = [s[i] for i in range(len(s)) if g[i]=="'Man'"]

    score_all.append(Counter(s).most_common()[0][0])
    if s_f:
        score_f.append(Counter(s_f).most_common()[0][0])
    else:
        score_f.append(np.nan)
    if s_m:
        score_m.append(Counter(s_m).most_common()[0][0])
    else:
        score_m.append(np.nan)